In [44]:
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup

def to_int(x):
    if type(x) is int:
        return x
    return int(x.replace(",","").replace(".",""))

In [45]:
url = "https://www.careerlink.vn/vieclam/list?keyword_use=A"
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')

total_jobs_r = soup.find("div", class_="col-md-7").get_text()

total_jobs = total_jobs_r.split()[2]
total_jobs = to_int(total_jobs)


In [46]:
url = "https://www.careerlink.vn/tim-viec-lam-nhanh"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify()

full_job_inds_dict = {}
full_ind = []

job_inds = soup.find('div', class_='masonry-container').find_all('li')

for ind in job_inds:
    job_ind = ind.find('a').get_text()
    full_ind.append(job_ind)
    job_ind_no = ind.find('span').get_text()
    job_ind_no = to_int(job_ind_no[1:-1])
    full_job_inds_dict[f'{job_ind}'] = job_ind_no

df1 = pd.DataFrame(full_job_inds_dict.items(), columns = ['Industry','No. of Jobs'])

df1

,Industry,No. of Jobs
0,Biên phiên dịch / Thông dịch viên,974
1,Biên phiên dịch (tiếng Nhật),594
2,Nhân sự,859
3,Pháp lý / Luật,195
4,Thư ký / Hành chánh,1851
5,An Ninh / Bảo Vệ,128
6,Bán lẻ / Bán sỉ,1172
7,Chăm sóc sức khỏe / Y tế,358
8,Dịch vụ khách hàng,2531
9,Phi chính phủ / Phi lợi nhuận,2


In [47]:
#cut last 5 rows of the dataframe and transpose them
#create new dataframe
#merge

url_list = ['https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=I',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=N',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=L',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=T',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=P',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=M',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=D',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=S',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=E'
           ]

senority_list = ['Intern', 'Mới đi làm', 'Nhân viên', 'Kỹ thuật viên', 'Trưởng nhóm', 'Quản lý', 'Giám đốc', 'Quản lí cấp cao', 'Điều hành cấp cao']
senority_no = []
senority_dict = {}

for url in url_list:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    senority_no_r = soup.find("div", class_="col-md-7").get_text()
    senority_no.append(to_int(senority_no_r.split()[2]))
    
senority_dict = dict(zip(senority_list, senority_no))
senority_dict
    


{'Intern': 157,
 'Mới đi làm': 422,
 'Nhân viên': 14636,
 'Kỹ thuật viên': 849,
 'Trưởng nhóm': 1404,
 'Quản lý': 1513,
 'Giám đốc': 174,
 'Quản lí cấp cao': 14,
 'Điều hành cấp cao': 5}

In [52]:
Careerlink = {'Total Jobs': total_jobs,
              'Industry': full_ind}

Careerlink.update(senority_dict)

df = DataFrame(Careerlink, columns=['Total Jobs', 'Intern', 'Mới đi làm', 'Nhân viên', 'Kỹ thuật viên', 
                                    'Trưởng nhóm', 'Quản lý', 'Giám đốc', 'Quản lí cấp cao', 'Điều hành cấp cao', 
                                    'Industry'])

Careerlink_result = df.merge(df1, on='Industry')

In [53]:
sum_column_1 = Careerlink_result["Mới đi làm"] + Careerlink_result["Nhân viên"] + Careerlink_result["Kỹ thuật viên"]
sum_column_2 = Careerlink_result["Trưởng nhóm"] + Careerlink_result["Quản lý"]
sum_column_3 = Careerlink_result["Giám đốc"] + Careerlink_result["Quản lí cấp cao"] + Careerlink_result["Điều hành cấp cao"]

Careerlink_result["Junior"] = sum_column_1
Careerlink_result["Senior"] = sum_column_2
Careerlink_result['Manager/Director'] = sum_column_3


In [54]:
Careerlink_result = Careerlink_result.drop(['Mới đi làm','Nhân viên','Kỹ thuật viên','Trưởng nhóm','Quản lý',
                                            'Giám đốc','Quản lí cấp cao', 'Điều hành cấp cao' ],axis=1)

Careerlink_result = Careerlink_result.reindex(columns=['Total Jobs','Intern','Junior','Senior',
                                                       'Manager/Director','Industry', 'No. of Jobs'])
Careerlink_result

,Total Jobs,Intern,Junior,Senior,Manager/Director,Industry,No. of Jobs
0,19174,157,15907,2917,193,Biên phiên dịch / Thông dịch viên,974
1,19174,157,15907,2917,193,Biên phiên dịch (tiếng Nhật),594
2,19174,157,15907,2917,193,Nhân sự,859
3,19174,157,15907,2917,193,Pháp lý / Luật,195
4,19174,157,15907,2917,193,Thư ký / Hành chánh,1851
5,19174,157,15907,2917,193,An Ninh / Bảo Vệ,128
6,19174,157,15907,2917,193,Bán lẻ / Bán sỉ,1172
7,19174,157,15907,2917,193,Chăm sóc sức khỏe / Y tế,358
8,19174,157,15907,2917,193,Dịch vụ khách hàng,2531
9,19174,157,15907,2917,193,Phi chính phủ / Phi lợi nhuận,2
